In [6]:
!pip install --proxy http://proxy-dmz.intel.com:911/ seaborn


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 KB 2.7 MB/s eta 0:00:00a 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [7]:
import numpy as np
import scipy.io as sio # for reading matlab files
import pathlib
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import zipfile

%matplotlib inline

In [11]:
root_dir = Path.cwd() # set the root directory as a Pathlib path
# data_dir = f'{root_dir}/data'
data_dir = root_dir / 'data/'
folder_raw_data = data_dir / 'raw_milling_uc_berkeley' # raw data folder that holds the .zip .mat files for milling data
folder_processed_data = data_dir / 'processed_milling_uc_berkeley/' # processed data folder
working_folder = data_dir / 'dev' # working folder
# folder_raw_data = f'{data_dir}/raw_milling_uc_berkeley/' # raw data folder that holds the .zip .mat files for milling data
# folder_processed_data = f'{data_dir}/processed_milling_uc_berkeley/'# processed data folder
# working_folder = f'{root_dir}/dev' # working folder

In [13]:
# if the the raw data folder does not exist, then you are likely
# in a google colab environment. In that case, we will create the 
# raw data and processed data folders and download the appropriate
# files

In [12]:
if folder_raw_data.exists() == False:

    pathlib.Path(folder_raw_data).mkdir(parents=True, exist_ok=True)
    os.chdir(folder_raw_data)
    !wget 'https://github.com/tvhahn/Manufacturing-Data-Science-with-Python/raw/master/Data%20Sets/milling_uc_berkeley/raw/mill.zip'

--2022-05-24 21:53:10--  https://github.com/tvhahn/Manufacturing-Data-Science-with-Python/raw/master/Data%20Sets/milling_uc_berkeley/raw/mill.zip
Resolving proxy-dmz.intel.com (proxy-dmz.intel.com)... 10.7.211.16
Connecting to proxy-dmz.intel.com (proxy-dmz.intel.com)|10.7.211.16|:912... connected.
Proxy request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tvhahn/Manufacturing-Data-Science-with-Python/master/Data%20Sets/milling_uc_berkeley/raw/mill.zip [following]
--2022-05-24 21:53:10--  https://raw.githubusercontent.com/tvhahn/Manufacturing-Data-Science-with-Python/master/Data%20Sets/milling_uc_berkeley/raw/mill.zip
Connecting to proxy-dmz.intel.com (proxy-dmz.intel.com)|10.7.211.16|:912... connected.
Proxy request sent, awaiting response... 200 OK
Length: 15140674 (14M) [application/zip]
Saving to: ‘mill.zip’

mill.zip            100%[===================>]  14.44M  9.85MB/s    in 1.5s    

2022-05-24 21:53:13 (9.85 MB/s) - ‘mill.zip’ saved [151406

In [14]:
if folder_processed_data.exists() == False:

    pathlib.Path(folder_processed_data).mkdir(parents=True, exist_ok=True)
    os.chdir(folder_processed_data)
    !wget 'https://raw.githubusercontent.com/tvhahn/Manufacturing-Data-Science-with-Python/master/Data%20Sets/milling_uc_berkeley/processed/labels_with_tool_class.csv'


--2022-05-24 21:53:57--  https://raw.githubusercontent.com/tvhahn/Manufacturing-Data-Science-with-Python/master/Data%20Sets/milling_uc_berkeley/processed/labels_with_tool_class.csv
Resolving proxy-dmz.intel.com (proxy-dmz.intel.com)... 10.7.211.16
Connecting to proxy-dmz.intel.com (proxy-dmz.intel.com)|10.7.211.16|:912... connected.
Proxy request sent, awaiting response... 200 OK
Length: 6449 (6.3K) [text/plain]
Saving to: ‘labels_with_tool_class.csv’

labels_with_tool_cl 100%[===================>]   6.30K  --.-KB/s    in 0.002s  

2022-05-24 21:53:57 (3.29 MB/s) - ‘labels_with_tool_class.csv’ saved [6449/6449]



In [15]:
# if working folder does not exist, then create it
pathlib.Path(working_folder).mkdir(parents=True, exist_ok=True)
if (working_folder / 'data_prep.py').exists() == False:
    os.chdir(working_folder)   
    # download important python files into the 'Metal Machining' directory
    !wget 'https://raw.githubusercontent.com/tvhahn/Manufacturing-Data-Science-with-Python/master/Metal%20Machining/data_prep.py'


--2022-05-24 21:54:19--  https://raw.githubusercontent.com/tvhahn/Manufacturing-Data-Science-with-Python/master/Metal%20Machining/data_prep.py
Resolving proxy-dmz.intel.com (proxy-dmz.intel.com)... 10.7.211.16
Connecting to proxy-dmz.intel.com (proxy-dmz.intel.com)|10.7.211.16|:912... connected.
Proxy request sent, awaiting response... 200 OK
Length: 11880 (12K) [text/plain]
Saving to: ‘data_prep.py’

data_prep.py        100%[===================>]  11.60K  --.-KB/s    in 0.02s   

2022-05-24 21:54:19 (685 KB/s) - ‘data_prep.py’ saved [11880/11880]



In [16]:
# extract mill.mat from the zip file
with zipfile.ZipFile(folder_raw_data / 'mill.zip', 'r') as zip_ref:
    zip_ref.extractall(folder_raw_data)

In [17]:
os.chdir(working_folder)

In [26]:
# load the data from the matlab file
m = sio.loadmat(folder_raw_data / 'mill.mat',struct_as_record=True)

In [27]:
# show some of the info from the matlab file
print('Keys in the matlab dict file: \n', m.keys(), '\n')

Keys in the matlab dict file: 
 dict_keys(['__header__', '__version__', '__globals__', 'mill']) 



In [28]:
# check to see what m['mill'] is
print(type(m['mill']))

# store the 'mill' data in a seperate np array
data = m['mill']

<class 'numpy.ndarray'>


In [29]:
# store the field names in the data np array in a tuple, l
fields = data.dtype.names
print('List of the field names:\n', fields)

List of the field names:
 ('case', 'run', 'VB', 'time', 'DOC', 'feed', 'material', 'smcAC', 'smcDC', 'vib_table', 'vib_spindle', 'AE_table', 'AE_spindle')
